<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllEkphrasisnegpos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('all')
device_name = tf.test.gpu_device_name()
print(device_name)
df_train = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/train_clean.csv',names=['Tweet','Emotion'])
df_test = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/test_clean.csv',names=['Tweet','Emotion'])
df_train = df_train.sample(frac=1)
df_test = df_train.sample(frac=1)

In [2]:
df_train.head(1000)

,Tweet,Emotion
5778,lizzidol i wish i wa in va too,2
10235,thi week ha been aw,2
10940,kaeci mine ben slow sinc last night think i mi...,2
11865,wow i am realli missin the famili today badd,2
10852,wa so excit to eat the wartermelon i bought th...,2
...,...,...
6432,huffi is not have a good day night or life poo...,2
5903,at work get a quick bite to eat befor have to ...,0
4337,stephenfri i did not get it although as it a r...,0
6486,realli good concept at mozconcept i realli lov...,2


In [3]:
df_test.Emotion.unique()

array(['0', '1', '2', '3', 'emotion'], dtype=object)

In [4]:
df_train.nunique()

Tweet      12370
Emotion        5
dtype: int64

In [5]:
category_dict = {'0':[1,0,0,0],'1':[0,1,0,0],'2':[0,0,1,0],'3':[0,0,0,1],'emotion':[1,0,0,0]}
train_data_tweet = [x.lower().split() for x in df_train['Tweet']]
train_data_cat = np.array([category_dict[x] for x in df_train['Emotion']])
test_data_tweet = [x.lower().split() for x in df_test['Tweet']]
test_data_cat = np.array([category_dict[x] for x in df_test['Emotion']])

data_tweet = train_data_tweet + test_data_tweet

In [6]:
# Parameters
Min_count = 0
Embedding_size = 100
Window_size = 5
Negative_sampling = 00

In [7]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)
w2v_sg.wv.init_sims(True)
w2v_cbow.wv.init_sims(True)

In [8]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.100d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [18]:
batch_size = 500
Max_input_size = max([len(x) for x in data_tweet])
mul_factor=1.5

In [10]:
positive = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/CodeMix/positive-words.csv')
positive = list(positive['0'][1:])
negative = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/CodeMix/negative-words.csv')
negative = list(negative['0'][2:])
pn_dict = {}
for i in range(len(positive)):
  pn_dict[positive[i]] = 1

for i in range(len(negative)):
  pn_dict[negative[i]] = -1

In [11]:
X_train = train_data_tweet
X_test = test_data_tweet
y_train = train_data_cat
y_test = test_data_cat 
X_train = [nltk.pos_tag(x) for x in X_train]
X_test = [nltk.pos_tag(x) for x in X_test]
X_train = [[(i[0],i[1],pn_dict.get(i[0],1)) for i in x] for x in X_train]
X_test = [[(i[0],i[1],pn_dict.get(i[0],1)) for i in x] for x in X_test]
result_table = [0,0,0,0,0,0,0,0,0]

In [12]:
print(X_train[0][0])

('lizzidol', 'NN', 1)


In [13]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(w2v.wv.get_vector(i[0]),i[1],i[2]) for i in x if i[0] in vocab]
        l=len(tweet)
        for i in range(l):
          tweet[i]=((tweet[i][0]*tweet[i][2]*mul_factor),tweet[i][1],tweet[i][2])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)          
      
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1],i[2]) for i in x if i[0] in vocab.keys() else]
        l=len(tweet)
        for i in range(l):
          tweet[i]=((tweet[i][0]*tweet[i][2]*mul_factor),tweet[i][1],tweet[i][2])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [23]:
epochs = 50

In [24]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(50))
model_sg.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_sg_LSTM = model_sg.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[0]=history[1]
print("SG_LSTM")


Epoch 1/50
24/24 [==============================] - 5s 225ms/step - loss: 1.2967 - accuracy: 0.5496
Epoch 2/50
24/24 [==============================] - 5s 226ms/step - loss: 1.0234 - accuracy: 0.5855
Epoch 3/50
24/24 [==============================] - 5s 228ms/step - loss: 0.9795 - accuracy: 0.5853
Epoch 4/50
24/24 [==============================] - 6s 240ms/step - loss: 0.9716 - accuracy: 0.5856
Epoch 5/50
24/24 [==============================] - 6s 248ms/step - loss: 0.9603 - accuracy: 0.5866
Epoch 6/50
24/24 [==============================] - 5s 229ms/step - loss: 0.9398 - accuracy: 0.5917
Epoch 7/50
24/24 [==============================] - 5s 228ms/step - loss: 0.9097 - accuracy: 0.6157
Epoch 8/50
24/24 [==============================] - 6s 229ms/step - loss: 0.8815 - accuracy: 0.6278
Epoch 9/50
24/24 [==============================] - 5s 227ms/step - loss: 0.8614 - accuracy: 0.6313
Epoch 10/50
24/24 [==============================] - 5s 227ms/step - loss: 0.8167 - accuracy: 0.6543

In [25]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_sg_bi = model_sg_bi.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[1]=history[1]
print("SG_BI")

Epoch 1/50
24/24 [==============================] - 6s 231ms/step - loss: 1.1969 - accuracy: 0.4890
Epoch 2/50
24/24 [==============================] - 6s 236ms/step - loss: 0.9823 - accuracy: 0.5831
Epoch 3/50
24/24 [==============================] - 6s 230ms/step - loss: 0.9294 - accuracy: 0.5937
Epoch 4/50
24/24 [==============================] - 6s 231ms/step - loss: 0.8877 - accuracy: 0.6067
Epoch 5/50
24/24 [==============================] - 5s 229ms/step - loss: 0.8416 - accuracy: 0.6377
Epoch 6/50
24/24 [==============================] - 6s 247ms/step - loss: 0.7865 - accuracy: 0.6730
Epoch 7/50
24/24 [==============================] - 6s 247ms/step - loss: 0.7324 - accuracy: 0.7123
Epoch 8/50
24/24 [==============================] - 5s 229ms/step - loss: 0.6816 - accuracy: 0.7335
Epoch 9/50
24/24 [==============================] - 6s 229ms/step - loss: 0.6164 - accuracy: 0.7687
Epoch 10/50
24/24 [==============================] - 6s 229ms/step - loss: 0.5621 - accuracy: 0.7969

In [26]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(50))
model_sg_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_sg_rnn = model_sg_rnn.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_rnn.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[2]=history[1]
print("SG_RNN")

Epoch 1/50
24/24 [==============================] - 6s 241ms/step - loss: 1.2145 - accuracy: 0.4473
Epoch 2/50
24/24 [==============================] - 6s 241ms/step - loss: 1.0010 - accuracy: 0.5780
Epoch 3/50
24/24 [==============================] - 6s 240ms/step - loss: 0.9857 - accuracy: 0.5839
Epoch 4/50
24/24 [==============================] - 6s 247ms/step - loss: 0.9792 - accuracy: 0.5853
Epoch 5/50
24/24 [==============================] - 6s 243ms/step - loss: 0.9732 - accuracy: 0.5863
Epoch 6/50
24/24 [==============================] - 6s 258ms/step - loss: 0.9662 - accuracy: 0.5882
Epoch 7/50
24/24 [==============================] - 6s 255ms/step - loss: 0.9594 - accuracy: 0.5903
Epoch 8/50
24/24 [==============================] - 6s 241ms/step - loss: 0.9515 - accuracy: 0.5914
Epoch 9/50
24/24 [==============================] - 6s 242ms/step - loss: 0.9430 - accuracy: 0.5928
Epoch 10/50
24/24 [==============================] - 6s 242ms/step - loss: 0.9340 - accuracy: 0.5945

In [27]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(50))
model_cbow.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_cbow_LSTM = model_cbow.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[3]=history[1]
print("CBOW_LSTM")

Epoch 1/50
24/24 [==============================] - 5s 225ms/step - loss: 1.2555 - accuracy: 0.5515
Epoch 2/50
24/24 [==============================] - 5s 225ms/step - loss: 1.0111 - accuracy: 0.5854
Epoch 3/50
24/24 [==============================] - 5s 223ms/step - loss: 0.9781 - accuracy: 0.5857
Epoch 4/50
24/24 [==============================] - 5s 226ms/step - loss: 0.9676 - accuracy: 0.5865
Epoch 5/50
24/24 [==============================] - 6s 234ms/step - loss: 0.9499 - accuracy: 0.5883
Epoch 6/50
24/24 [==============================] - 6s 243ms/step - loss: 0.9228 - accuracy: 0.5985
Epoch 7/50
24/24 [==============================] - 6s 237ms/step - loss: 0.8831 - accuracy: 0.6226
Epoch 8/50
24/24 [==============================] - 5s 224ms/step - loss: 0.8396 - accuracy: 0.6435
Epoch 9/50
24/24 [==============================] - 5s 224ms/step - loss: 0.8113 - accuracy: 0.6543
Epoch 10/50
24/24 [==============================] - 5s 225ms/step - loss: 0.7895 - accuracy: 0.6666

In [28]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_cbow_bi = model_cbow_bi.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[4]=history[1]
print("CBOW_BI")

Epoch 1/50
24/24 [==============================] - 6s 236ms/step - loss: 1.2301 - accuracy: 0.4700
Epoch 2/50
24/24 [==============================] - 6s 234ms/step - loss: 0.9924 - accuracy: 0.5830
Epoch 3/50
24/24 [==============================] - 6s 230ms/step - loss: 0.9280 - accuracy: 0.5940
Epoch 4/50
24/24 [==============================] - 5s 226ms/step - loss: 0.8919 - accuracy: 0.6058
Epoch 5/50
24/24 [==============================] - 6s 230ms/step - loss: 0.8512 - accuracy: 0.6310
Epoch 6/50
24/24 [==============================] - 5s 229ms/step - loss: 0.7971 - accuracy: 0.6681
Epoch 7/50
24/24 [==============================] - 5s 229ms/step - loss: 0.7371 - accuracy: 0.7063
Epoch 8/50
24/24 [==============================] - 5s 225ms/step - loss: 0.6731 - accuracy: 0.7391
Epoch 9/50
24/24 [==============================] - 5s 226ms/step - loss: 0.6099 - accuracy: 0.7720
Epoch 10/50
24/24 [==============================] - 5s 225ms/step - loss: 0.5606 - accuracy: 0.7914

In [29]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(50))
model_cbow_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_cbow_rnn = model_cbow_rnn.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_rnn.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[5]=history[1]
print("CBOW_RNN")

Epoch 1/50
24/24 [==============================] - 6s 243ms/step - loss: 1.2006 - accuracy: 0.4803
Epoch 2/50
24/24 [==============================] - 6s 246ms/step - loss: 1.0041 - accuracy: 0.5786
Epoch 3/50
24/24 [==============================] - 6s 233ms/step - loss: 0.9874 - accuracy: 0.5833
Epoch 4/50
24/24 [==============================] - 6s 235ms/step - loss: 0.9823 - accuracy: 0.5844
Epoch 5/50
24/24 [==============================] - 6s 243ms/step - loss: 0.9774 - accuracy: 0.5854
Epoch 6/50
24/24 [==============================] - 6s 233ms/step - loss: 0.9718 - accuracy: 0.5874
Epoch 7/50
24/24 [==============================] - 6s 235ms/step - loss: 0.9672 - accuracy: 0.5885
Epoch 8/50
24/24 [==============================] - 6s 232ms/step - loss: 0.9613 - accuracy: 0.5913
Epoch 9/50
24/24 [==============================] - 6s 233ms/step - loss: 0.9558 - accuracy: 0.5937
Epoch 10/50
24/24 [==============================] - 6s 230ms/step - loss: 0.9495 - accuracy: 0.5972

In [35]:
epochs=100
Embedding_size=100

In [40]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove.add(tf.keras.layers.LSTM(50))
model_glove.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_glove_LSTM = model_glove.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[6]=history[1]
print("GLOVE_LSTM")


Epoch 1/150
24/24 [==============================] - 1s 47ms/step - loss: 1.2074 - accuracy: 0.5767
Epoch 2/150
24/24 [==============================] - 1s 48ms/step - loss: 1.0010 - accuracy: 0.5851
Epoch 3/150
24/24 [==============================] - 1s 47ms/step - loss: 0.9858 - accuracy: 0.5853
Epoch 4/150
24/24 [==============================] - 1s 47ms/step - loss: 0.9823 - accuracy: 0.5854
Epoch 5/150
24/24 [==============================] - 1s 47ms/step - loss: 0.9777 - accuracy: 0.5854
Epoch 6/150
24/24 [==============================] - 1s 50ms/step - loss: 0.9674 - accuracy: 0.5855
Epoch 7/150
24/24 [==============================] - 1s 46ms/step - loss: 0.9545 - accuracy: 0.5857
Epoch 8/150
24/24 [==============================] - 1s 46ms/step - loss: 0.9424 - accuracy: 0.5885
Epoch 9/150
24/24 [==============================] - 1s 46ms/step - loss: 0.9293 - accuracy: 0.5920
Epoch 10/150
24/24 [==============================] - 1s 46ms/step - loss: 0.9140 - accuracy: 0.5996

In [38]:
epochs=150
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_glove_bi = model_glove_bi.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[7]=history[1]
print("GLOVE_BI")

Epoch 1/150
24/24 [==============================] - 1s 47ms/step - loss: 1.1887 - accuracy: 0.4950
Epoch 2/150
24/24 [==============================] - 1s 49ms/step - loss: 0.9873 - accuracy: 0.5804
Epoch 3/150
24/24 [==============================] - 1s 48ms/step - loss: 0.9555 - accuracy: 0.5926
Epoch 4/150
24/24 [==============================] - 1s 49ms/step - loss: 0.9352 - accuracy: 0.5972
Epoch 5/150
24/24 [==============================] - 1s 48ms/step - loss: 0.9193 - accuracy: 0.6033
Epoch 6/150
24/24 [==============================] - 1s 48ms/step - loss: 0.9028 - accuracy: 0.6097
Epoch 7/150
24/24 [==============================] - 1s 49ms/step - loss: 0.8846 - accuracy: 0.6166
Epoch 8/150
24/24 [==============================] - 1s 50ms/step - loss: 0.8644 - accuracy: 0.6296
Epoch 9/150
24/24 [==============================] - 1s 48ms/step - loss: 0.8514 - accuracy: 0.6352
Epoch 10/150
24/24 [==============================] - 1s 48ms/step - loss: 0.8353 - accuracy: 0.6446

In [39]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(50))
model_glove_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_rnn.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_rnn.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[8]=history[1]
print("GLOVE_RNN")

Epoch 1/150
24/24 [==============================] - 2s 65ms/step - loss: 1.1880 - accuracy: 0.4678
Epoch 2/150
24/24 [==============================] - 1s 61ms/step - loss: 1.0028 - accuracy: 0.5818
Epoch 3/150
24/24 [==============================] - 1s 60ms/step - loss: 0.9895 - accuracy: 0.5845
Epoch 4/150
24/24 [==============================] - 1s 60ms/step - loss: 0.9858 - accuracy: 0.5851
Epoch 5/150
24/24 [==============================] - 1s 61ms/step - loss: 0.9814 - accuracy: 0.5852
Epoch 6/150
24/24 [==============================] - 1s 59ms/step - loss: 0.9774 - accuracy: 0.5853
Epoch 7/150
24/24 [==============================] - 1s 60ms/step - loss: 0.9752 - accuracy: 0.5855
Epoch 8/150
24/24 [==============================] - 1s 61ms/step - loss: 0.9728 - accuracy: 0.5858
Epoch 9/150
24/24 [==============================] - 1s 62ms/step - loss: 0.9697 - accuracy: 0.5860
Epoch 10/150
24/24 [==============================] - 1s 58ms/step - loss: 0.9666 - accuracy: 0.5859

In [ ]:
model_sg1 = tf.keras.Sequential()
model_sg1.add(tf.keras.Input(shape=(Max_input_size,100)))
model_sg1.add(tf.keras.layers.LSTM(64))
model_sg1.add(tf.keras.layers.Dropout(0.2))
model_sg1.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg1.add(tf.keras.layers.Dropout(0.2))
model_sg1.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg1.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg1.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg1.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("SG_LSTM")

Epoch 1/40
24/24 [==============================] - 6s 230ms/step - loss: 1.2186 - accuracy: 0.5771
Epoch 2/40
24/24 [==============================] - 6s 241ms/step - loss: 1.0215 - accuracy: 0.5858
Epoch 3/40
24/24 [==============================] - 6s 238ms/step - loss: 1.0042 - accuracy: 0.5857
Epoch 4/40
24/24 [==============================] - 6s 235ms/step - loss: 0.9958 - accuracy: 0.5863
Epoch 5/40
24/24 [==============================] - 6s 235ms/step - loss: 0.9933 - accuracy: 0.5863
Epoch 6/40
24/24 [==============================] - 6s 236ms/step - loss: 0.9941 - accuracy: 0.5863
Epoch 7/40
24/24 [==============================] - 6s 236ms/step - loss: 0.9916 - accuracy: 0.5863
Epoch 8/40
24/24 [==============================] - 6s 235ms/step - loss: 0.9918 - accuracy: 0.5863
Epoch 9/40
24/24 [==============================] - 6s 233ms/step - loss: 0.9917 - accuracy: 0.5863
Epoch 10/40
24/24 [==============================] - 6s 235ms/step - loss: 0.9907 - accuracy: 0.5863

In [ ]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])

	Lstm			BiLstm			Rnn
sg [0.5845000147819519, 0.5845000147819519, 0.5845000147819519]
cbow [0.5845000147819519, 0.5845000147819519, 0.5845000147819519]
glove [0.750333309173584, 0.7973333597183228, 0.6434166431427002]


In [ ]:
print(result_table[9])

0.5845000147819519
